In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install segmentation_models_pytorch 
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 37.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 KB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 23.3 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16444 sha256=21b56f7214f02e02970895ddf3ecf0f92a4aad00f93d3ba6295b0d7f0ce5304b
  Stored in directory: /root/.cache/pip/wheels/29/16/24/752e89d88d333af39a288421e64d613b5f652918e39ef1f8e3
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60962 sha256=8349a7a47c6bb5b84f326d93aba5d14bdbad9582f84399e9c8f9025676f8032f
  Stored in directory: /root/.cache/pip/w

In [ ]:
from collections import namedtuple
import os
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.io as tv
import torchvision.transforms as transf
import torch.utils.data
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
from torch.optim import Adam
from tqdm import tqdm
import segmentation_models_pytorch as smp
import torchmetrics
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
# For reproducibility
torch.manual_seed(100)
if torch.cuda.is_available():
    torch.cuda.manual_seed(100)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

#################################################################################################################################
# Label nemedtuple
# taken from Dataset offical Github: https://github.com/mcordts/cityscapesScripts/blob/master/cityscapesscripts/helpers/labels.py
#################################################################################################################################
Label = namedtuple( 'Label' , [

    'name'        , # The identifier of this label, e.g. 'car', 'person', ... .
                    # We use them to uniquely name a class

    'id'          , # An integer ID that is associated with this label.
                    # The IDs are used to represent the label in ground truth images
                    # An ID of -1 means that this label does not have an ID and thus
                    # is ignored when creating ground truth images (e.g. license plate).
                    # Do not modify these IDs, since exactly these IDs are expected by the
                    # evaluation server.

    'trainId'     , # Feel free to modify these IDs as suitable for your method. Then create
                    # ground truth images with train IDs, using the tools provided in the
                    # 'preparation' folder. However, make sure to validate or submit results
                    # to our evaluation server using the regular IDs above!
                    # For trainIds, multiple labels might have the same ID. Then, these labels
                    # are mapped to the same class in the ground truth images. For the inverse
                    # mapping, we use the label that is defined first in the list below.
                    # For example, mapping all void-type classes to the same ID in training,
                    # might make sense for some approaches.
                    # Max value is 255!

    'category'    , # The name of the category that this label belongs to

    'categoryId'  , # The ID of this category. Used to create ground truth images
                    # on category level.

    'hasInstances', # Whether this label distinguishes between single instances or not

    'ignoreInEval', # Whether pixels having this class as ground truth label are ignored
                    # during evaluations or not

    'color'       , # The color of this label
    ] )

labels = [
    #       name                     id    trainId   category            catId     hasInstances   ignoreInEval   color
    Label(  'unlabeled'            ,  0 ,      0 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'ego vehicle'          ,  1 ,      0 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'rectification border' ,  2 ,      0 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'out of roi'           ,  3 ,      0 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'static'               ,  4 ,      0 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'dynamic'              ,  5 ,      0 , 'void'            , 0       , False        , True         , (111, 74,  0) ),
    Label(  'ground'               ,  6 ,      0 , 'void'            , 0       , False        , True         , ( 81,  0, 81) ),
    Label(  'road'                 ,  7 ,      1 , 'flat'            , 1       , False        , False        , (128, 64,128) ),
    Label(  'sidewalk'             ,  8 ,      2 , 'flat'            , 1       , False        , False        , (244, 35,232) ),
    Label(  'parking'              ,  9 ,      0 , 'flat'            , 1       , False        , True         , (250,170,160) ),
    Label(  'rail track'           , 10 ,      0 , 'flat'            , 1       , False        , True         , (230,150,140) ),
    Label(  'building'             , 11 ,      3 , 'construction'    , 2       , False        , False        , ( 70, 70, 70) ),
    Label(  'wall'                 , 12 ,      4 , 'construction'    , 2       , False        , False        , (102,102,156) ),
    Label(  'fence'                , 13 ,      5 , 'construction'    , 2       , False        , False        , (190,153,153) ),
    Label(  'guard rail'           , 14 ,      0 , 'construction'    , 2       , False        , True         , (180,165,180) ),
    Label(  'bridge'               , 15 ,      0 , 'construction'    , 2       , False        , True         , (150,100,100) ),
    Label(  'tunnel'               , 16 ,      0 , 'construction'    , 2       , False        , True         , (150,120, 90) ),
    Label(  'pole'                 , 17 ,      6 , 'object'          , 3       , False        , False        , (153,153,153) ),
    Label(  'polegroup'            , 18 ,      0 , 'object'          , 3       , False        , True         , (153,153,153) ),
    Label(  'traffic light'        , 19 ,      7 , 'object'          , 3       , False        , False        , (250,170, 30) ),
    Label(  'traffic sign'         , 20 ,      8 , 'object'          , 3       , False        , False        , (220,220,  0) ),
    Label(  'vegetation'           , 21 ,      9 , 'nature'          , 4       , False        , False        , (107,142, 35) ),
    Label(  'terrain'              , 22 ,     10 , 'nature'          , 4       , False        , False        , (152,251,152) ),
    Label(  'sky'                  , 23 ,     11 , 'sky'             , 5       , False        , False        , ( 70,130,180) ),
    Label(  'person'               , 24 ,     12 , 'human'           , 6       , True         , False        , (220, 20, 60) ),
    Label(  'rider'                , 25 ,     13 , 'human'           , 6       , True         , False        , (255,  0,  0) ),
    Label(  'car'                  , 26 ,     14 , 'vehicle'         , 7       , True         , False        , (  0,  0,142) ),
    Label(  'truck'                , 27 ,     15 , 'vehicle'         , 7       , True         , False        , (  0,  0, 70) ),
    Label(  'bus'                  , 28 ,     16 , 'vehicle'         , 7       , True         , False        , (  0, 60,100) ),
    Label(  'caravan'              , 29 ,      0 , 'vehicle'         , 7       , True         , True         , (  0,  0, 90) ),
    Label(  'trailer'              , 30 ,      0 , 'vehicle'         , 7       , True         , True         , (  0,  0,110) ),
    Label(  'train'                , 31 ,     17 , 'vehicle'         , 7       , True         , False        , (  0, 80,100) ),
    Label(  'motorcycle'           , 32 ,     18 , 'vehicle'         , 7       , True         , False        , (  0,  0,230) ),
    Label(  'bicycle'              , 33 ,     19 , 'vehicle'         , 7       , True         , False        , (119, 11, 32) ),
    Label(  'license plate'        , -1 ,      0 , 'vehicle'         , 7       , False        , True         , (  0,  0,142) ),
]

#################################################################################################################################
# Define custom Train and Validation set classes
# This assusmes that you have downloaded the Cityscpes dataset, and placed the masks with your training IDs in the meta folder
# Method inspired from: https://github.com/fregu856/deeplabv3/blob/master/datasets.py
#################################################################################################################################
train_dirs = ["jena/", "zurich/", "weimar/", "ulm/", "tubingen/", "stuttgart/",
              "strasbourg/", "monchengladbach/", "krefeld/", "hanover/",
              "hamburg/", "erfurt/", "dusseldorf/", "darmstadt/", "cologne/",
              "bremen/"]
val_dirs = ["frankfurt/", "munster/", "lindau/"]
test_dirs = ["bochum/", "aachen/"]

cityscapes_data_path = '../content/drive/MyDrive/Cityscapes_data'
cityscapes_meta_path = '../content/drive/MyDrive/Cityscapes_data/meta'

class DatasetTrain(torch.utils.data.Dataset):
  def __init__(self, cityscapes_data_path, cityscapes_meta_path):
    self.img_dir = cityscapes_data_path + "/leftImg8bit/train/"
    self.label_dir = cityscapes_meta_path + "/label_imgs/"

    self.examples = []

    for train_dir in train_dirs:
      train_img_dir_path = self.img_dir + train_dir

      file_names = os.listdir(train_img_dir_path)
      for file_name in file_names:
        img_id = file_name.split("_leftImg8bit.png")[0]
        img_path = train_img_dir_path + file_name
        label_img_path = self.label_dir + img_id + ".png"

        example = {}
        example["img_path"] = img_path
        example["label_img_path"] = label_img_path
        example["img_id"] = img_id
        self.examples.append(example)

    self.num_examples = len(self.examples)

  def __len__(self):
    return self.num_examples

  def __getitem__(self, index):
    example = self.examples[index]

    img_path = example["img_path"]
    transform = transf.Resize((256,512), antialias=True)
    img = tv.read_image(img_path)
    img = transform(img)
    img = img.to(torch.float32) / 255.0

    label_img_path = example["label_img_path"]
    label_img = tv.read_image(label_img_path)
    label_img = transform(label_img)
    label_img = label_img.to(torch.float32)

    return (img, label_img)

class DatasetVal(torch.utils.data.Dataset):
  def __init__(self, cityscapes_data_path, cityscapes_meta_path):
    self.img_dir = cityscapes_data_path + "/leftImg8bit/val/"
    self.label_dir = cityscapes_meta_path + "/label_imgs/"

    self.examples = []

    for val_dir in val_dirs:
      val_img_dir_path = self.img_dir + val_dir

      file_names = os.listdir(val_img_dir_path)
      for file_name in file_names:
        img_id = file_name.split("_leftImg8bit.png")[0]
        img_path = val_img_dir_path + file_name
        label_img_path = self.label_dir + img_id + ".png"

        example = {}
        example["img_path"] = img_path
        example["label_img_path"] = label_img_path
        example["img_id"] = img_id
        self.examples.append(example)

    self.num_examples = len(self.examples)

  def __len__(self):
    return self.num_examples

  def __getitem__(self, index):
    example = self.examples[index]

    img_path = example["img_path"]
    
    transform = transf.Resize((256,512), antialias=True)
    img = tv.read_image(img_path)
    img = transform(img)
    img = img.to(torch.float32) / 255.0

    label_img_path = example["label_img_path"]
    label_img = tv.read_image(label_img_path)
    label_img = transform(label_img)
    label_img = label_img.to(torch.float32)

    return (img, label_img)

train_dataset = DatasetTrain(cityscapes_data_path=cityscapes_data_path,
                             cityscapes_meta_path=cityscapes_meta_path)

val_dataset = DatasetVal(cityscapes_data_path=cityscapes_data_path,
                         cityscapes_meta_path=cityscapes_meta_path)

#################################################################################################################################
# Define UNET-CBAM
#################################################################################################################################
# CBAM inspired form official CBAM Github: https://github.com/Jongchan/attention-module
class Flatten(nn.Module):
  def forward(self, x):
    return x.view(x.size(0), -1)

class ChannelGate(nn.Module):
  def __init__(self, gate_channels, reduction_ratio=16):
    super(ChannelGate, self).__init__()
    self.gate_channels = gate_channels
    self.mlp = nn.Sequential(Flatten(),
                             nn.Linear(gate_channels, gate_channels // reduction_ratio),
                             nn.ReLU(),
                             nn.Linear(gate_channels // reduction_ratio, gate_channels))

  def forward(self, x):
    x_avg_pool = F.avg_pool2d(x, (x.size(2), x.size(3)), stride=(x.size(2), x.size(3)))
    x_max_pool = F.max_pool2d(x, (x.size(2), x.size(3)), stride=(x.size(2), x.size(3)))
    x_avg_att = self.mlp(x_avg_pool)
    x_max_att = self.mlp(x_max_pool)
    channel_att = x_avg_att + x_max_att
    scale = torch.sigmoid(channel_att).unsqueeze(2).unsqueeze(3).expand_as(x)
    return x * scale

class ChannelPool(nn.Module):
    def forward(self, x):
        return torch.cat((torch.max(x,1)[0].unsqueeze(1), torch.mean(x,1).unsqueeze(1)), dim=1 )

class BasicConv(nn.Module):
  def __init__(self, in_planes, out_planes, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=False):
    super(BasicConv, self).__init__()
    self.out_channels = out_planes
    self.conv = nn.Conv2d(in_planes, out_planes, kernel_size=kernel_size, stride=stride, padding=padding, dilation=dilation, groups=groups, bias=bias)
    self.bn = nn.BatchNorm2d(out_planes,eps=1e-5, momentum=0.01, affine=True) 

  def forward(self, x):
    x = self.conv(x)
    x = self.bn(x)
    return x

class SpatialGate(nn.Module):
  def __init__(self):
    super(SpatialGate, self).__init__()
    kernel_size = 7
    self.compress = ChannelPool()
    self.spatial = BasicConv(2, 1, kernel_size, stride=1, padding=(kernel_size-1) // 2)

  def forward(self, x):
    x_compress = self.compress(x)
    x_out = self.spatial(x_compress)
    scale = torch.sigmoid(x_out) # broadcasting
    return x * scale

class CBAM(nn.Module):
  def __init__(self, gate_channels, reduction_ratio=16):
    super(CBAM, self).__init__()
    self.ChannelGate = ChannelGate(gate_channels, reduction_ratio)
    self.SpatialGate = SpatialGate()

  def forward(self, x):
    x_out = self.ChannelGate(x)
    x_out = self.SpatialGate(x_out)
    return x_out

# DoubleConv to be used by UNET
class DoubleConv(nn.Module):
  def __init__(self, in_channels, out_channels):
    super(DoubleConv, self).__init__()
    self.conv = nn.Sequential(
      nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
      nn.BatchNorm2d(out_channels),
      nn.ReLU(inplace=True),
      nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
      nn.BatchNorm2d(out_channels),
      nn.ReLU(inplace=True),
      )
    
  def forward(self, x):
    return self.conv(x)

class UNet(nn.Module):
  def __init__(self, in_channels, out_channels, inter_channels=[64, 128, 256, 512]):
    super(UNet, self).__init__()
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

    # Encoder - Contracting path (down path)
    self.e1 = DoubleConv(in_channels, inter_channels[0])
    self.cbam1 = CBAM(gate_channels=inter_channels[0])
    self.e2 = DoubleConv(inter_channels[0], inter_channels[1])
    self.cbam2 = CBAM(gate_channels=inter_channels[1])
    self.e3 = DoubleConv(inter_channels[1], inter_channels[2])
    self.cbam3 = CBAM(gate_channels=inter_channels[2])
    self.e4 = DoubleConv(inter_channels[2], inter_channels[3])
    self.cbam4 = CBAM(gate_channels=inter_channels[3])

    # Bottom part of the network
    self.bottom = DoubleConv(inter_channels[-1], inter_channels[-1]*2)
    self.cbam_bottom = CBAM(gate_channels=inter_channels[-1]*2)

    # Decoder - Expansive path (up path)
    rev_channels = inter_channels[::-1]
    self.d4_T = nn.ConvTranspose2d(rev_channels[0]*2, rev_channels[0], kernel_size=2, stride=2, padding=0)
    self.d4_C = DoubleConv(rev_channels[0]*2, rev_channels[0])
    self.cbam4_d = CBAM(gate_channels=rev_channels[0])
    self.d3_T = nn.ConvTranspose2d(rev_channels[0], rev_channels[1], kernel_size=2, stride=2, padding=0)
    self.d3_C = DoubleConv(rev_channels[0], rev_channels[1])
    self.cbam3_d = CBAM(gate_channels=rev_channels[1])
    self.d2_T = nn.ConvTranspose2d(rev_channels[1], rev_channels[2], kernel_size=2, stride=2, padding=0)
    self.d2_C = DoubleConv(rev_channels[1], rev_channels[2])
    self.cbam2_d = CBAM(gate_channels=rev_channels[2])
    self.d1_T = nn.ConvTranspose2d(rev_channels[2], rev_channels[3], kernel_size=2, stride=2, padding=0)
    self.d1_C = DoubleConv(rev_channels[2], rev_channels[3])
    self.cbam1_d = CBAM(gate_channels=rev_channels[3])

    # Final convolution
    self.output = nn.Conv2d(rev_channels[3], out_channels, kernel_size=1)
    self.softmax = nn.Softmax(dim=1)

  def forward(self, x):
    x1 = self.e1(x)
    x1 = self.cbam1(x1)
    x = self.pool(x1)
    x2 = self.e2(x)
    x2 = self.cbam2(x2)
    x = self.pool(x2)
    x3 = self.e3(x)
    x3 = self.cbam3(x3)
    x = self.pool(x3)
    x4 = self.e4(x)
    x4 = self.cbam4(x4)
    x = self.pool(x4)
    x_bottom = self.bottom(x)
    x_bottom = self.cbam_bottom(x_bottom)
    x = self.d4_T(x_bottom)
    x = torch.cat((x4, x), dim=1)
    x = self.d4_C(x)
    x = self.cbam4_d(x)
    x = self.d3_T(x)
    x = torch.cat((x3, x), dim=1)
    x = self.d3_C(x)
    x = self.cbam3_d(x)
    x = self.d2_T(x)
    x = torch.cat((x2, x), dim=1)
    x = self.d2_C(x)
    x = self.cbam2_d(x)
    x = self.d1_T(x)
    x = torch.cat((x1, x), dim=1)
    x = self.d1_C(x)
    x = self.cbam1_d(x)
    x = self.output(x)
    return x

#################################################################################################################################
# Define data loader, load model, hyper-parameters and metrics
#################################################################################################################################
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# model
model = UNet(3, 20, [64, 128, 256, 512]).to(device)
# Loss Function
n_classes = 20
loss_fn = nn.CrossEntropyLoss()
# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, min_lr= 0.00125, verbose=True)
lr = []

jaccard_micro = torchmetrics.JaccardIndex(task="multiclass", num_classes=20, average="micro").to(device)
jaccard_macro = torchmetrics.JaccardIndex(task="multiclass", num_classes=20, average="macro").to(device)
jaccard_weighted = torchmetrics.JaccardIndex(task="multiclass", num_classes=20, average="weighted").to(device)
jaccard_none = torchmetrics.JaccardIndex(task="multiclass", num_classes=20, average=None).to(device)

epochs = 50

len_train = len(train_loader)
len_val = len(val_loader)

train_loss_per_epoch_avg = []
train_jaccard_micro_per_epoch_avg = []
train_jaccard_macro_per_epoch_avg = []
train_jaccard_weighted_per_epoch_avg = []
train_jaccard_none_per_epoch_avg = []

val_loss_per_epoch_avg = []
val_jaccard_micro_per_epoch_avg = []
val_jaccard_macro_per_epoch_avg = []
val_jaccard_weighted_per_epoch_avg = []
val_jaccard_none_per_epoch_avg = []

best_val_loss = float('inf')

folder_path = "/content/drive/MyDrive/Cityscapes_data/256x512_4_batches/unet_cbam_full/"

#################################################################################################################################
# Training and validation loops
#################################################################################################################################
for e in range(epochs):
  train_loss_total = 0
  train_jaccard_micro_total=0
  train_jaccard_macro_total=0
  train_jaccard_weighted_total=0
  train_jaccard_none_total=0

  val_loss_total = 0
  val_jaccard_micro_total=0
  val_jaccard_macro_total=0
  val_jaccard_weighted_total=0
  val_jaccard_none_total=0

  model.train()

  print("Epoch: {}".format(e+1))

  # training
  for batch, (image, label) in enumerate(tqdm(train_loader)):
    optimizer.zero_grad()
    image = image.to(device)
    label = label.to(device)
    output = model(image)

    loss = loss_fn(output.to(torch.float32), torch.squeeze(label, 1).long())
    loss.backward()
    optimizer.step()
    train_loss_total += loss.item()

    x = torch.softmax(output, dim=1)
    index = torch.argmax(x, dim=1)
    y = torch.unsqueeze(index, dim=1)
    train_jaccard_micro_total += jaccard_micro(y, label)
    train_jaccard_macro_total += jaccard_macro(y, label)
    train_jaccard_weighted_total += jaccard_weighted(y, label)
    train_jaccard_none_total += jaccard_none(y, label)
    
  train_loss_per_epoch_avg.append(train_loss_total/len_train)
  print("Epoch: {}, Average train loss per epoch: {}".format(e+1, train_loss_per_epoch_avg[e]))
  train_jaccard_micro_per_epoch_avg.append(train_jaccard_micro_total/len_train)
  print("Epoch: {}, Average train jaccard_micro coeff per epoch: {}".format(e+1, train_jaccard_micro_per_epoch_avg[e]))
  train_jaccard_macro_per_epoch_avg.append(train_jaccard_macro_total/len_train)
  print("Epoch: {}, Average train jaccard_macro coeff per epoch: {}".format(e+1, train_jaccard_macro_per_epoch_avg[e]))
  train_jaccard_weighted_per_epoch_avg.append(train_jaccard_weighted_total/len_train)
  print("Epoch: {}, Average train jaccard_weighted coeff per epoch: {}".format(e+1, train_jaccard_weighted_per_epoch_avg[e]))
  train_jaccard_none_per_epoch_avg.append(train_jaccard_none_total/len_train)
  print("Epoch: {}, Average train jaccard_none coeff per epoch: {}".format(e+1, train_jaccard_none_per_epoch_avg[e]))

  train_loss_total = 0
  train_jaccard_micro_total = 0
  train_jaccard_macro_total = 0
  train_jaccard_weighted_total = 0
  train_jaccard_none_total = 0

  torch.save(model, folder_path + "city_unet.pth")
  file = open(folder_path + 'train_loss.txt','w')
  for item in train_loss_per_epoch_avg:
	  file.write(str(item)+"\n")
  file.close()

  torch.save(train_jaccard_micro_per_epoch_avg, folder_path + "train_jaccard_micro_per_epoch_avg.pth")
  torch.save(train_jaccard_macro_per_epoch_avg, folder_path + "train_jaccard_macro_per_epoch_avg.pth")
  torch.save(train_jaccard_weighted_per_epoch_avg, folder_path + "train_jaccard_weighted_per_epoch_avg.pth")
  torch.save(train_jaccard_none_per_epoch_avg, folder_path + "train_jaccard_none_per_epoch_avg.pth")


  # validating
  with torch.no_grad():
    model.eval()
    for batch, (image, label) in enumerate(tqdm(val_loader)):
      image = image.to(device)
      label = label.to(device)
      output = model(image)

      loss = loss_fn(output.to(torch.float32), torch.squeeze(label, 1).long())
      val_loss_total += loss.item()

      x = torch.softmax(output, dim=1)
      index = torch.argmax(x, dim=1)
      y = torch.unsqueeze(index, dim=1)
      val_jaccard_micro_total += jaccard_micro(y, label)
      val_jaccard_macro_total += jaccard_macro(y, label)
      val_jaccard_weighted_total += jaccard_weighted(y, label)
      val_jaccard_none_total += jaccard_none(y, label)

  scheduler.step(val_loss_total)
  print('epoch={}, learning rate={:.4f}'.format(e, optimizer.state_dict()['param_groups'][0]['lr']))
  lr.append(optimizer.state_dict()['param_groups'][0]['lr'])

  if val_loss_total < best_val_loss:
    torch.save(model.state_dict(), folder_path + 'best_weights.pth')
    best_val_loss = val_loss_total

  val_loss_per_epoch_avg.append(val_loss_total/len_val)
  print("Epoch: {}, Average val loss per epoch: {}".format(e+1, val_loss_per_epoch_avg[e]))
  val_jaccard_micro_per_epoch_avg.append(val_jaccard_micro_total/len_val)
  print("Epoch: {}, Average val jaccard_micro coeff per epoch: {}".format(e+1, val_jaccard_micro_per_epoch_avg[e]))
  val_jaccard_macro_per_epoch_avg.append(val_jaccard_macro_total/len_val)
  print("Epoch: {}, Average val jaccard_macro coeff per epoch: {}".format(e+1, val_jaccard_macro_per_epoch_avg[e]))
  val_jaccard_weighted_per_epoch_avg.append(val_jaccard_weighted_total/len_val)
  print("Epoch: {}, Average val jaccard_weighted coeff per epoch: {}".format(e+1, val_jaccard_weighted_per_epoch_avg[e]))
  val_jaccard_none_per_epoch_avg.append(val_jaccard_none_total/len_val)
  print("Epoch: {}, Average val jaccard_none coeff per epoch: {}".format(e+1, val_jaccard_none_per_epoch_avg[e]))

  val_loss_total = 0
  val_jaccard_micro_total = 0
  val_jaccard_macro_total = 0
  val_jaccard_weighted_total = 0
  val_jaccard_none_total = 0

  file = open(folder_path + 'val_loss.txt','w')
  for item in val_loss_per_epoch_avg:
	  file.write(str(item)+"\n")
  file.close()

  file = open(folder_path + 'lr.txt','w')
  for item in lr:
	  file.write(str(item)+"\n")
  file.close()

  torch.save(val_jaccard_micro_per_epoch_avg, folder_path + "val_jaccard_micro_per_epoch_avg.pth")
  torch.save(val_jaccard_macro_per_epoch_avg, folder_path + "/val_jaccard_macro_per_epoch_avg.pth")
  torch.save(val_jaccard_weighted_per_epoch_avg, folder_path + "/val_jaccard_weighted_per_epoch_avg.pth")
  torch.save(val_jaccard_none_per_epoch_avg, folder_path + "/val_jaccard_none_per_epoch_avg.pth")

Epoch: 1


100%|██████████| 170/170 [50:39<00:00, 17.88s/it]


Epoch: 1, Average train loss per epoch: 1.1978999383309308
Epoch: 1, Average train jaccard_micro coeff per epoch: 0.4859926104545593
Epoch: 1, Average train jaccard_macro coeff per epoch: 0.15018637478351593
Epoch: 1, Average train jaccard_weighted coeff per epoch: 0.4746137261390686
Epoch: 1, Average train jaccard_none coeff per epoch: tensor([3.5821e-01, 6.9922e-01, 7.3583e-02, 4.7262e-01, 1.9466e-05, 8.1105e-05,
        3.4397e-05, 1.9446e-05, 2.1356e-05, 5.5467e-01, 8.6543e-02, 5.0416e-01,
        5.0816e-06, 3.6412e-06, 2.5453e-01, 0.0000e+00, 3.6874e-06, 0.0000e+00,
        0.0000e+00, 2.2318e-06], device='cuda:0')


100%|██████████| 32/32 [09:25<00:00, 17.66s/it]


epoch=0, learning rate=0.0100
Epoch: 1, Average val loss per epoch: 1.5467418171465397
Epoch: 1, Average val jaccard_micro coeff per epoch: 0.37382030487060547
Epoch: 1, Average val jaccard_macro coeff per epoch: 0.12437404692173004
Epoch: 1, Average val jaccard_weighted coeff per epoch: 0.41063910722732544
Epoch: 1, Average val jaccard_none coeff per epoch: tensor([0.4852, 0.5480, 0.0248, 0.4089, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.4684, 0.0666, 0.3757, 0.0000, 0.0000, 0.1097, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000], device='cuda:0')
Epoch: 2


...

epoch=48, learning rate=0.0013
Epoch: 49, Average val loss per epoch: 0.5724103013053536
Epoch: 49, Average val jaccard_micro coeff per epoch: 0.754052460193634
Epoch: 49, Average val jaccard_macro coeff per epoch: 0.39428871870040894
Epoch: 49, Average val jaccard_weighted coeff per epoch: 0.7711252570152283
Epoch: 49, Average val jaccard_none coeff per epoch: tensor([0.6391, 0.8997, 0.5960, 0.7892, 0.1232, 0.1254, 0.3211, 0.1012, 0.2345,
        0.8494, 0.3025, 0.8116, 0.3749, 0.0655, 0.8295, 0.1474, 0.1983, 0.0669,
        0.0390, 0.3713], device='cuda:0')
Epoch: 50


100%|██████████| 170/170 [05:51<00:00,  2.06s/it]


Epoch: 50, Average train loss per epoch: 0.257033372714239
Epoch: 50, Average train jaccard_micro coeff per epoch: 0.8627684116363525
Epoch: 50, Average train jaccard_macro coeff per epoch: 0.5315767526626587
Epoch: 50, Average train jaccard_weighted coeff per epoch: 0.86970055103302
Epoch: 50, Average train jaccard_none coeff per epoch: tensor([0.8528, 0.9696, 0.8199, 0.8744, 0.2934, 0.3874, 0.3660, 0.1460, 0.3163,
        0.8804, 0.5393, 0.8952, 0.5469, 0.1496, 0.9038, 0.4231, 0.3666, 0.2400,
        0.1649, 0.4959], device='cuda:0')


100%|██████████| 32/32 [00:58<00:00,  1.82s/it]

epoch=49, learning rate=0.0013
Epoch: 50, Average val loss per epoch: 0.5644780546426773
Epoch: 50, Average val jaccard_micro coeff per epoch: 0.7505946755409241
Epoch: 50, Average val jaccard_macro coeff per epoch: 0.39438769221305847
Epoch: 50, Average val jaccard_weighted coeff per epoch: 0.7693668603897095
Epoch: 50, Average val jaccard_none coeff per epoch: tensor([0.6326, 0.8950, 0.5892, 0.7898, 0.1257, 0.1141, 0.3199, 0.1114, 0.2441,
        0.8470, 0.3043, 0.8176, 0.3880, 0.0749, 0.8305, 0.1354, 0.2073, 0.0604,
        0.0217, 0.3789], device='cuda:0')
